In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import optuna

from category_encoders.target_encoder import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import gpboost as gpb

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Set print options
np.set_printoptions(suppress=True, precision=6, edgeitems = 7)
pd.options.display.float_format = '{:.6f}'.format
pd.set_option('display.max_columns', None)

Notebook objective: Tune & test GPBoost model with LGBM booster only, without store_id, without random effects component, to see if Booster works properly & learns considerably.

# Preprocessing

In [3]:
random_state = 1923

In [4]:
# Read data
df = pd.read_csv("./InputData/full_data.csv")

In [5]:
# Reindex data from 0:N
df = df.reset_index(drop = True)

In [6]:
# Drop rows with too high duration
high_end = 10800 # 3 hours
df = df[df["duration"] <= high_end]

In [7]:
# Split features & target, drop non-feature columns
X = df.drop(["created_at", "actual_delivery_time", "duration"], axis = 1)
y = df.duration

In [8]:
# Split features & group variables
#G = X["store_id"]
X = X.drop(["store_id"], axis = 1)

In [9]:
train_end = int(len(df) * 0.6)

In [10]:
val_end = train_end + int(len(df) * 0.2)

In [11]:
# Train - val - test split, 60 - 20 - 20
X_train, X_val, X_test = X[:train_end], X[train_end:val_end], X[val_end:]
#G_train, G_val, G_test = G[:train_end], G[train_end:val_end], G[val_end:]
y_train, y_val, y_test = y[:train_end], y[train_end:val_end], y[val_end:]

In [12]:
# Create target encoders

# store_id encoder with hierarchy, top level market_id
#hierarchy = pd.DataFrame(X["market_id"]).rename({"market_id": "HIER_store_id_1"}, axis = 1)
#encoder_storeid = TargetEncoder(cols = ["store_id"], hierarchy = hierarchy)

# Encoder for remaining categoricals, without hierarchy
encoder = TargetEncoder(cols = ["market_id", "store_primary_category", "order_protocol"])

pipeline = Pipeline([
    #("encoder_storeid", encoder_storeid),
    ("encoder", encoder)
])

In [13]:
# Preprocess data
X_train = encoder.fit_transform(X_train, y_train)
X_val = encoder.transform(X_val)
X_test = encoder.transform(X_test)

# Hyperparameter tuning

In [14]:
# Objective function
def objective_gpb(trial):

    # Define hyperparameter space
    learning_rate = trial.suggest_float("learning_rate", 0.05, 0.5)
    num_leaves = trial.suggest_int("num_leaves", 2**2, 2**10)
    #max_depth = trial.suggest_int("max_depth", 2, 20) # Max depth of 20 is too restrictive for LightGBM
    min_child_samples = trial.suggest_int("min_child_samples", 10, 1000, log = True)
    min_child_weight = trial.suggest_float("min_child_weight", 0.001, 20, log = True)
    reg_alpha = trial.suggest_float("l1_reg", 0, 1)
    reg_lambda = trial.suggest_float("l2_reg", 0, 2)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.25, 1)

    # Create model
    #callbacks = [gpb.early_stopping(50)]
    
    model = gpb.GPBoostRegressor(
        n_jobs = 10,
        #device_type = "gpu",
        n_estimators = 5000,
        num_leaves = num_leaves,
        random_state = random_state,
        #max_depth = max_depth,
        max_depth = -1,
        min_child_samples = min_child_samples,
        learning_rate = learning_rate,
        min_child_weight = min_child_weight,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        colsample_bytree = colsample_bytree
    )

    # Train model with early stopping
    model.fit(
        X_train, 
        y_train,
        eval_set = [(X_val, y_val)],
        early_stopping_rounds = 50,
        #callbacks = callbacks,
        verbose = False)

    # Report best number of rounds
    trial.set_user_attr("n_rounds", (model.best_iteration_))
    
    return model.best_score_['valid_0']['l2']


In [15]:
# Create study
study_gpb = optuna.create_study(
  sampler = optuna.samplers.TPESampler(seed = random_state),
  study_name = "tune_gpb",
  direction = "minimize"
)

[I 2023-11-30 10:06:26,205] A new study created in memory with name: tune_gpb


In [16]:
# Perform study
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_gpb.optimize(
  objective_gpb, 
  n_trials = 1000,
  show_progress_bar = True)

Best trial: 873. Best value: 682757: 100%|█████████████████████████████████████████| 1000/1000 [10:06<00:00,  1.65it/s]


In [18]:
# Save tuning log
trials_gpb = study_gpb.trials_dataframe().sort_values("value", ascending = True)
trials_gpb.to_csv("./ModifiedData/trials_gpb_boostonly.csv", index = False)

# Testing & diagnostics

In [20]:
# Load best tune
best_tune = pd.read_csv("./ModifiedData/trials_gpb_boostonly.csv").iloc[0]

In [21]:
best_tune

number                                             873
value                                    682756.682359
datetime_start              2023-11-30 10:14:43.784578
datetime_complete           2023-11-30 10:14:44.662729
duration                        0 days 00:00:00.878151
params_colsample_bytree                       0.326927
params_l1_reg                                 0.866363
params_l2_reg                                 0.006419
params_learning_rate                          0.061448
params_min_child_samples                            32
params_min_child_weight                       0.484801
params_num_leaves                                  141
user_attrs_n_rounds                                185
state                                         COMPLETE
Name: 0, dtype: object

In [22]:
# Combine train & validation data
X_train, X_test = X[:val_end], X[val_end:]
#G_train, G_test = G[:val_end], G[val_end:]
y_train, y_test = y[:val_end], y[val_end:]

In [23]:
# Preprocess data
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

In [24]:
# Create gpb data
train = gpb.Dataset(X_train, y_train)
test = gpb.Dataset(X_test, y_test)

In [25]:
# Create params dict
params = {
    "random_state": random_state,
    "n_estimators": int(best_tune["user_attrs_n_rounds"]),
    "num_leaves": best_tune["params_num_leaves"],
    "min_child_samples": best_tune["params_min_child_samples"],
    "learning_rate": best_tune["params_learning_rate"],
    "min_child_weight": best_tune["params_min_child_weight"],
    "reg_alpha": best_tune["params_l1_reg"],
    "reg_lambda": best_tune["params_l2_reg"],
    "colsample_bytree": best_tune["params_colsample_bytree"]
}

In [26]:
# Train booster
model = gpb.train(
    params = params,
    train_set = train,
    #gp_model = gp_model
)

Found `n_estimators` in params. Will use it instead of argument


[GPBoost] [Info] Total Bins 2872
[GPBoost] [Info] Number of data points in the train set: 88345, number of used features: 28
[GPBoost] [Info] Start training from score 2816.359805


In [27]:
# Make predictions on test data
preds = model.predict(X_test)

In [28]:
preds

array([4180.274705, 3504.973232, 3893.982999, 3317.624105, 4041.577925,
       3361.262114, 2428.790867, ..., 1865.030631, 2085.736266,
       1506.938441, 2057.773275, 2716.021282, 2862.154006, 2531.72598 ])

In [29]:
# Combine back with test data
df_pred = X_test.copy()
df_pred["preds"] = preds
df_pred["actual"] = y_test
df_pred["residual"] = y_test - preds

In [30]:
# Calculate RMSE, MAPE
print("RMSE:")
print(mean_squared_error(df_pred["actual"], df_pred["preds"], squared = False))

print("MAPE:")
print(mean_absolute_percentage_error(df_pred["actual"], df_pred["preds"]))

RMSE:
922.72800488589
MAPE:
0.22985026092589383


# Findings
- Performs very similar to 4.4LGBM.
- Booster works & learns as expected, and tunes have a significant impact on model performance.
- The issue must come from combining the booster & the random effects component.